In [27]:
import mediapipe as mp
import numpy as np
import screen_brightness_control as control
import cv2
from math import hypot

In [31]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_draw = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)

while(True):
    r,frame = cap.read()

    if r == True:
        frame = cv2.flip(frame,1)
        rgb_frame = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        frame = cv2.resize(frame,(750,750))

        results = hands.process(rgb_frame)
        landmark_list = []

        if results.multi_hand_landmarks:
            for hand_landmark in results.multi_hand_landmarks:
                for id,lm in enumerate(hand_landmark.landmark):
                    height,width,_ = frame.shape
                    centre_x = int(lm.x * width)
                    centre_y = int(lm.y * height)
                    landmark_list.append([id,centre_x,centre_y])
                    
                    mp_draw.draw_landmarks(frame,hand_landmark,mp_hands.HAND_CONNECTIONS)

        if landmark_list:
            x1,y1 = landmark_list[4][1],landmark_list[4][2]
            x2,y2 = landmark_list[8][1],landmark_list[8][2]

            cv2.circle(img = frame,center = (x1,y1),radius = 5,color = (0,255,0),thickness = cv2.FILLED)
            cv2.circle(img = frame,center = (x2,y2),radius = 5,color = (0,255,0),thickness = cv2.FILLED)
            cv2.line(frame,(x1,y1),(x2,y2),(0,0,255),5)

            euclidean_length = hypot(x2-x1,y2-y1)
            brightness = np.interp(euclidean_length, [10,220],[0,100])

            control.set_brightness(int(brightness))

        cv2.imshow("LIVE",frame)
        if cv2.waitKey(1) & 0xff == ord("p"):
            break

cv2.destroyAllWindows()
